In [ ]:
%%capture
!uv pip install s3fs zarr cftime fsspec rioxarray

In [ ]:
%%capture
!wget -O data_utils.py https://raw.githubusercontent.com/Thomasjkeel/UKCEH_Summer_School/refs/heads/main/Workshop_4/data_utils.py

*Workshop 4. Doing research with hydrological data*


# Practical: Comparison of Rainfall Data: Gauges vs Gridded Estimates


## Learning objectives:
- Understand how to read in, format and analyse rainfall data from different sources
- Calculate and interpret basic statistics to compare gridded rainfall estimates and rain gauge observations 
- Learn how to test and scale up research code

# Introduction
Rainfall is notoriously difficult to measure accurately, but in the UK we have a dense network of both [rain gauges](https://en.wikipedia.org/wiki/Rain_gauge) and [rain radar](https://en.wikipedia.org/wiki/Weather_radar) stations (**Figure 1**). These network gives us a relatively reliable and consistent estimate of rainfall through time, even when there is missing or wrong data, because estimates are cross-checked agaisnt other nearby measurements. 

![Map of UK rain gauge stations](https://raw.githubusercontent.com/NERC-CEH/UKCEH_Summer_School/refs/heads/main/content/Maps-of-daily-rain-gauges-used-to-derive-the-CEH-GEAR-data-set-a-monthly-rain-gauges.png)  
*Figure 1. Maps of daily rain gauges in 2012: (a) monthly rain gauges and (b) daily rain gauges (source: Keller et al. 2015).*


The UKCEH is also working-on a updated interactive map of the UK's rain gauge network (as of 2025), which you can find a version of [here](https://thomasjkeel.github.io/UK-Rain-Gauge-Network/gauges.html).

## Gridded rainfall data
> *Gridded rainfall products provide a spatially- and temporally-uniform estimate of rainfall*

Whilst, the UK's rain gauge network provides valuable data, its coverage is both spatially and temporally uneven. Some areas—such as Scotland—have a much denser concentration of gauges than others, like the South of England (see Figure 1). Additionally, the gauges themselves vary in age and data quality.  

As such, it can be inconvenient to work with rain gauge data. For this reason, environmental researchers often prefer datasets that are interpolated onto regular grids, which are easier to process, analyze, and compare over space and time.


For the UK, two main gridded rainfall products exist:  
1. CEH-GEAR — provides daily/monthly 1km gridded rainfall, produced by the UKCEH and described in [Keller et al. 2015](https://essd.copernicus.org/articles/7/143/2015/)
2. HadUK-Grid — provides daily/monthly/seasonal/annual 1km gridded rainfall, produced by the Met Office and described in [Hollis et al. 2019](https://rmets.onlinelibrary.wiley.com/doi/full/10.1002/gdj3.78)

Each product provides slightly different methodology for spatially interpolating data from the UK rain gauge network to a regular 1km grid, shown diagramatically in Figure 2 (see more in those papers if you are interested in the specifics). For this practical, we will be using the CEH-GEAR dataset, but for future, you can access the HadUK-Grid [here](https://catalogue.ceda.ac.uk/uuid/4dc8450d889a491ebb20e724debe2dfb/)).


![example of gridding](https://raw.githubusercontent.com/NERC-CEH/UKCEH_Summer_School/refs/heads/main/content/gridding_example_from_gdal.png)  
*Figure 2. Diagram showing example of interpolation of points to regular grid (source: GDAL 2025).*


In this practical, we will read in both gridded rainfall and rain gauge data and compare them...



In [ ]:
# Load required libraries
import fsspec
import zarr
import rioxarray

import geopandas as gpd
import polars as pl
import xarray as xr
import seaborn as sns

import matplotlib.pyplot as plt
import shapely.geometry

# practical-specific file:
import data_utils

In [ ]:
# Global variables for coordinates of Severn River catchment
SEVERN_NORTHING_RANGE = [270000, 349000]
SEVERN_EASTING_RANGE = [280000, 390000]

# 1. Load rainfall data
First we will read in gridded rainfall and rain gauge data using code similar to that shown in Workshop 2.

**External data sources:**
- UK-wide CEH-GEAR rainfall data - from the JASMIN object-store
- Rain gauge data for the Severn River Catchment - from the JASMIN object-store
- Severn catchment boundaries - from the National River Flow Archive ([NRFA](https://nrfa.ceh.ac.uk/data/search))
- River Severn watercourse - from [OS Open Rivers](https://www.ordnancesurvey.co.uk/products/os-open-rivers)

## 1.1 Load CEH-GEAR (gridded rainfall data)
Using the code from Workshop 2, below we connect to the S3 bucket on the JASMIN object-store (remote) and load in a local version of the daily CEH-GEAR file ("geardaily_fulloutput_yearly_100km_chunks.zarr")

In [ ]:
fdri_fs = fsspec.filesystem(
    "s3", asynchronous=True, anon=True, endpoint_url="https://fdri-o.s3-ext.jc.rl.ac.uk"
)
gear_daily_zstore = zarr.storage.FsspecStore(
    fdri_fs, path="geardaily/GB/geardaily_fulloutput_yearly_100km_chunks.zarr"
)
gear_daily = xr.open_zarr(gear_daily_zstore, decode_times=True, decode_cf=True)
gear_daily  # 310 GB worth of data

In these practicals, there will be questions, please fill them in the best you can before moving on (or ask us for help)
##### ❓ Question: what type of file is the input data?
Replace the ??? in this markdown box with your answer 
(*hint: check the file extension*)

**Answer:** ???

##### ❓ Question: what data type is the 'gear_daily' data we have loaded in?

Replace the ??? in the code-block below with your answer

In [ ]:
type(???)

Let's have a quick look at one day in the CEH-GEAR dataset...

In [ ]:
# plot data 
# may take about 50 seconds
fig, ax = plt.subplots(1)
gear_daily.sel(time="2013-02-13")["rainfall_amount"].plot(ax=ax, cmap="Blues")

Remember, this figure shown daily rainfall for the entire UK from CEH-GEAR product. The CEH-GEAR spatially interpolates rain gauge observations to a regular 1km by 1km grid

## 1.2 Load daily rain gauge data
Next, we will load daily rain gauge data and it's associated metadata from the JASMIN object-store.

🐻‍❄️ **Note:** we use the [polars](https://pola.rs/) instead of pandas to load data, but its syntax should be relatively familiar.

In [ ]:
severn_rain_gauge_data = pl.read_csv(
    "s3://rain-gauge/hourly_severn_rain_gauge_data.csv",
    storage_options={"endpoint_url": "https://fdri-o.s3-ext.jc.rl.ac.uk", "anon": True},
    try_parse_dates=True,
)
severn_rain_gauge_metadata = pl.read_csv(
    "s3://rain-gauge/hourly_severn_rain_gauge_metadata.csv",
    storage_options={"endpoint_url": "https://fdri-o.s3-ext.jc.rl.ac.uk", "anon": True},
)

In [ ]:
severn_rain_gauge_data.head()

In [ ]:
severn_rain_gauge_metadata.head()

##### ❓ Question: what type of file is the input data?

**Answer:** ???

##### ❓ Question: what data type is the 'severn_rain_gauge_data'?

Replace the ??? in the code-block below with your answer

In [ ]:
???

Let's subset the data to look at precipitation from a single rain gauge...

In [ ]:
severn_rain_gauge_data.filter(pl.col("ID") == 90537)

🐼: above code is equivalent to: `severn_rain_gauge_data.loc[severn_rain_gauge_data["ID"] == 90537]`

#### 🤨 Task: Subset the precipitation data to the station: "WALFORD" 

Replace the ??? below with your answer

*Hint: you can use the `.filter` method like above*

In [ ]:
# Step 1. Find the "ID" for the station with the name: "WALFORD" in the metadata
severn_rain_gauge_metadata.???

In [ ]:
# Step 2. Using that ID, subset the rain gauge data
severn_rain_gauge_data.???

## 1.3 Load spatial datasets for Upper Severn catchment
Finally, we download some spatial data for the River Severn, including catchment boundaries from the NRFA, and watercourse data from Ordinance Survey.

Data originally downloaded from: [NRFA](https://nrfa.ceh.ac.uk/data/search) & [Ordinance Survey](https://www.ordnancesurvey.co.uk/products/os-open-rivers)

This bash script below will fetch and download spatial data from GitHub and store it locally in the folder 'severn_catchment_data'

In [2]:
%%capture
# !wget -O https://raw.githubusercontent.com/NERC-CEH/UKCEH_Summer_School/refs/heads/main/Workshop_4/severn_catchment_data.tar.gz
!mkdir -p severn_catchment_data
!tar -xvf severn_catchment_data.tar.gz

Load shapefiles & geojson using geopandas

In [ ]:
# Greater Severn catchment boundary
bewdley_shp = gpd.read_file("severn_catchment_data/Bewdley/54001/54001.shp")

# Upper Severn catchment boundaries
abermule_shp = gpd.read_file("severn_catchment_data/Abermule/54014/54014.shp")
dolwen_shp = gpd.read_file("severn_catchment_data/Dolwen/54080/54080.shp")
plynlimon_shp = gpd.read_file("severn_catchment_data/Plynlimon Flume/54022/54022.shp")

In [ ]:
# Load rivers linestrings
severn_catchment_river_linestrings = gpd.read_file(
    "severn_catchment_data/rivers_around_severn.geojson"
)
severn_river_linestrings = severn_catchment_river_linestrings.loc[
    severn_catchment_river_linestrings["name1"].str.contains("River Severn")
]

Next, we plot the catchment boundaries and the River Severn

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 6), sharex=True, sharey=True)
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
abermule_shp.plot(ax=ax, facecolor="none", alpha=0.5, linewidth=0.5)
bewdley_shp.plot(ax=ax, facecolor="none", alpha=0.5, linewidth=0.5)
dolwen_shp.plot(ax=ax, facecolor="none", alpha=0.5, linewidth=0.5)
plynlimon_shp.plot(ax=ax, facecolor="none", alpha=0.5, linewidth=0.5)

severn_catchment_river_linestrings.plot(ax=ax, alpha=0.1)
severn_river_linestrings.plot(ax=ax, alpha=0.6)
ax.set_title("Upper parts of the River Severn")
plt.subplots_adjust(hspace=0.2)

We'll also load in some height data and plot that

In [ ]:
severn_hght = rioxarray.open_rasterio("severn_catchment_data/HGHT_SEVERN_1KM_CLIP.tif")
severn_hght = severn_hght.sortby("y")
severn_hght = severn_hght.sel(band=1)
severn_hght = severn_hght / 10  # divide by 10, so in metres
severn_hght = severn_hght.sel(
    x=slice(*SEVERN_EASTING_RANGE), y=slice(*SEVERN_NORTHING_RANGE)
)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 6), sharex=True, sharey=True)
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
abermule_shp.plot(ax=ax, facecolor="none", linewidth=0.5)
bewdley_shp.plot(ax=ax, facecolor="none", linewidth=0.5)
dolwen_shp.plot(ax=ax, facecolor="none", linewidth=0.5)
plynlimon_shp.plot(ax=ax, facecolor="none", linewidth=0.5)
severn_hght.plot(ax=ax, alpha=0.7, cbar_kwargs={"label": "height (m)"})

ax.set_title("Height profile of upper parts of the River Severn")

# 2. Format the data
Most of data science is about data cleaning and formatting. This is especially true for those using environmental data, where we often have datasets from many different sources, and that are of varying quality.

In this section, we will prepare the datasets, so we can compare the rain gauge dataset to gridded rainfall in the next section.


## 2.1 Checking for gaps in rain observation data
There are many issues with the rain gauge observation dataset we are using because it has not been quality controlled. This includes gaps, streaks of repeating values, exceedances of world records, misinputted data, etc.. However, in the interest of moving swiftly on, we will only focus on finding gaps in this practical. To do this we calculate a data 'completeness' for each gauge comparing number of rows to expected number of rows.

>*Note:* if you are interested in quality controlling rain gauge data, see the Python package [RainfallQC](https://github.com/NERC-CEH/RainfallQC) and find some examples [here](https://github.com/Thomasjkeel/RainfallQC-notebooks)


In [ ]:
# To save RAM later on, we'll filter out rows before 1990
severn_rain_gauge_data = severn_rain_gauge_data.filter(
    pl.col("DATETIME") > pl.date(year=1990, month=1, day=1)
)

In [ ]:
# load in a single gappy gauge
one_gauge = severn_rain_gauge_data.filter(pl.col("ID") == 90806)

In [ ]:
# plot data to see gaps
one_gauge.to_pandas().set_index("DATETIME")[
    "PRECIPITATION"
].plot()  # here we convert polars to pandas to let us plot longer time series

In the above plot, clearly there are some gaps in the data. We do not want to include this in our final clean dataset.  

Let's create a completeness metric, by comparing the start and end dates of each gauge, and then checking how many data rows are missing

In [ ]:
# Compute observed number of days between start and end date (i.e. this is the number of observations in the data)
severn_rain_gauge_data_observed = severn_rain_gauge_data.group_by("ID").agg(
    [
        pl.col("DATETIME").min().alias("start_date"),
        pl.col("DATETIME").max().alias("end_date"),
        pl.len().alias("observed_days"),
    ]
)
severn_rain_gauge_data_observed.head()

In [ ]:
# Compute expected number of days between start and end date
severn_rain_gauge_data_observed_and_expected = (
    severn_rain_gauge_data_observed.with_columns(
        [
            ((pl.col("end_date") - pl.col("start_date")).dt.total_days() + 1).alias(
                "expected_days"
            ),
        ]
    )
)

severn_rain_gauge_data_observed_and_expected.head()

In [ ]:
# Divide observed by expected and get percentage completeness
severn_rain_gauge_data_completeness = (
    severn_rain_gauge_data_observed_and_expected.with_columns(
        [
            ((pl.col("observed_days") / pl.col("expected_days")) * 100).alias(
                "perc_completeness"
            )
        ]
    )
)

severn_rain_gauge_data_completeness.head()

Some of the rain gauges are particularly gappy...  
Remember 60 % completeness, means that only 60% of the days between the start and end date actually have a record for rainfall (maybe someone forgot on the other days)

In [ ]:
severn_rain_gauge_data_completeness.filter(pl.col("perc_completeness") < 60)

Always a good idea to use a histogram to view the distribution of your data too...

In [ ]:
fig, ax = plt.subplots(1)
sns.histplot(severn_rain_gauge_data_completeness["perc_completeness"], ax=ax)
ax.set_ylabel("Number of gauges")
ax.set_xlabel("Completeness (%)")
ax.grid()

Hmm, there are only a few gauges with very gappy data, let's remove everything below 90% just to be safe

### 🤨 Task: Filter out data with less than 90% completeness
Let's create a dataset that is more reliable

Replace the ??? below with your answer

*Hint: remember to break down the problem into steps, and print things out in a new code block if unsure*

In [ ]:
# Step 1. Get the IDs of gauges above 90% threshold
reliable_gauge_ids = severn_rain_gauge_data_completeness.filter(
    pl.col(???) > ???
)["ID"].to_list()

In [ ]:
# filter data to include only those ID's with more than 90% completeness
severn_rain_gauge_data_filtered = severn_rain_gauge_data.???(
    pl.col("ID").is_in(???)
)

#### 🤨 Optional task: Take a closer look at the gauge 428554, what other issues can you see that will need to be quality controlled?:


# 2.2 Subset spatial data

We have data for the entire UK, let's subset our data into the areas around the River Severn only and for data since 1990 (to make this notebook run quicker)

In [ ]:
# subset CEH-GEAR to SEVERN CATCHMENT
severn_gear_daily = gear_daily.sel(
    x=slice(*SEVERN_EASTING_RANGE),
    y=slice(
        *sorted(SEVERN_NORTHING_RANGE, reverse=True)
    ),  # has to be reversed as y coordinates is descending
    time=slice("1990", None),
)
severn_gear_daily = severn_gear_daily.drop_vars("min_dist")  # drop unecessary variable
severn_gear_daily = severn_gear_daily.sortby("y")  # flip y coordinates

In [ ]:
%%time
# Load data into RAM (memory)
# this code-block takes about 45 seconds
severn_gear_daily.load()

You can also see the spatial profile of the rain gauge data in the below plot. There are huge gaps towards the east of the Bewdley catchment, but we will ignore these in this practical.

In [ ]:
fig, ax = plt.subplots(1)
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
abermule_shp.plot(ax=ax, facecolor="none", alpha=0.8, linewidth=0.5)
bewdley_shp.plot(ax=ax, facecolor="none", alpha=0.8, linewidth=0.5)
dolwen_shp.plot(ax=ax, facecolor="none", alpha=0.8, linewidth=0.5)
plynlimon_shp.plot(ax=ax, facecolor="none", alpha=0.8, linewidth=0.5)
sns.scatterplot(
    x="EASTING",
    y="NORTHING",
    data=severn_rain_gauge_metadata.filter(pl.col("ID").is_in(reliable_gauge_ids)),
)
ax.set_title("Distribution of rain gauges in Upper Severn")

# 3. Compare gridded rainfall product (CEH-GEAR) with rain gauge data
Okay, now we can start running some analysis. Let's calculate some simple statistics of difference between gridded rainfall estimates and rain gauge observations.


## 3.1 Join one gauge to one grid cell
First, let's look at how we can join a single rain gauge to its nearest grid cell in the CEH-GEAR

In [ ]:
CHOSEN_GAUGE_ID = 90358

In [ ]:
# filter the important metadata
one_gauge = severn_rain_gauge_data_filtered.filter(pl.col("ID") == CHOSEN_GAUGE_ID)
one_gauge_metadata = severn_rain_gauge_metadata.filter(pl.col("ID") == CHOSEN_GAUGE_ID)
one_gauge_completeness = severn_rain_gauge_data_completeness.filter(
    pl.col("ID") == CHOSEN_GAUGE_ID
)

In [ ]:
one_gauge_metadata  # ah, 90358 is SUGNALL HALL and it is at a height of 142 meters

We can use the `method='nearest'` argument to get the nearest grid cell

In [ ]:
# spatial subset (get nearest grid cell)
one_gauge_grid_cell = severn_gear_daily.sel(
    x=one_gauge_metadata["EASTING"], y=one_gauge_metadata["NORTHING"], method="nearest"
)

# time subset (get data that overlaps in time)
one_gauge_grid_cell = one_gauge_grid_cell.sel(
    time=slice(
        one_gauge_completeness["start_date"].item(),
        one_gauge_completeness["end_date"].item(),
    )
)

In [ ]:
# plot both time series on the same y-axis
fig, axes = plt.subplots(2, 1, figsize=(9, 7), sharex=True, sharey=True)

one_gauge.to_pandas().set_index("DATETIME")["PRECIPITATION"].plot(ax=axes[0])
one_gauge_grid_cell["rainfall_amount"].plot(ax=axes[1])

axes[0].set_title(f"Rain gauge (ID={CHOSEN_GAUGE_ID})")
axes[1].set_title("Closest CEH-GEAR grid-cell")

for ax in axes:
    ax.grid()
    ax.set_ylabel("Precipitation (mm)")

They look a bit similar, but is hard to tell because this is a daily record.  
Let's more directly compare the two...

In [ ]:
# extract data from the nearest grid cell
one_gauge_grid_cell_df = pl.DataFrame(
    {
        "DATETIME": one_gauge_grid_cell["time"].data.flatten(),
        "PRECIPITATION_GRID": one_gauge_grid_cell["rainfall_amount"].data.flatten(),
    }
)

# This is needed to get datetime in correct format
one_gauge_grid_cell_df = one_gauge_grid_cell_df.with_columns(
    pl.col("DATETIME").cast(pl.Datetime("us"))
)

In [ ]:
# Join the data together
one_gauge_joined = one_gauge.join(one_gauge_grid_cell_df, on="DATETIME", how="left")
one_gauge_joined = one_gauge_joined.rename({"PRECIPITATION": "PRECIPITATION_GAUGE"})
one_gauge_joined.head()

##### ❓ Question: The two rainfall records seem similar, but are they correlated?
*Hint: see cell below*  
**Answer:** ???

In [ ]:
# Run a pearson Correlation
one_gauge_joined.select(
    pl.corr("PRECIPITATION_GAUGE", "PRECIPITATION_GRID", method="pearson")
)

Let's compute a difference through time

In [ ]:
one_gauge_joined = one_gauge_joined.with_columns(
    (pl.col("PRECIPITATION_GAUGE") - pl.col("PRECIPITATION_GRID")).alias(
        "rainfall_diff"
    )
)

In [ ]:
one_gauge_joined.head()

In [ ]:
# drop null values so that we are not including values where either the CEH-GEAR or rain gauge is a missing value
one_gauge_joined = one_gauge_joined.drop_nulls()

In [ ]:
one_gauge_joined.to_pandas().set_index("DATETIME")["rainfall_diff"].plot()
plt.title(
    f"Difference between gauge {CHOSEN_GAUGE_ID} & the nearest CEH-GEAR grid cell"
)
plt.ylabel("Rainfall difference (mm)")

In [ ]:
print("mean difference =", one_gauge_joined["rainfall_diff"].mean())

Looks like the variability of the rainfall difference might change through after around 2009, let's look quickly look at the standard deviation before and after 2009

In [ ]:
rainfall_difference_before2000 = one_gauge_joined.filter(pl.col("DATETIME") < pl.date(year=2009, month=1, day=1))['rainfall_diff']
rainfall_difference_after2009 = one_gauge_joined.filter(pl.col("DATETIME") >= pl.date(year=2009, month=1, day=1))['rainfall_diff']

In [ ]:
print(f"Standard deviation of difference 1990-2009: {rainfall_difference_stdev_before2000.std()}")
print(f"Standard deviation in difference 2009-2019: {rainfall_difference_stdev_after2009.std()}")

It also looks like there may be an decrease in the number of days where the estimate difference is above -/+ 1 mm
#### 🤨 Optional Task: Use the `filter` method to get the number of days where the rainfall difference is -/+ 1 and then count the number of days each year

Replace the ??? below with your answer

*Hint: because you are looking for +/- 1, it is a good idea to use absolute numbers i.e. abs()*  
*Hint: group_by_dynamic*

In [3]:
# ???

## 3.1.1 Look at monthly running total
To make the plot more visually appealing, we can compute the difference in rainfall based on monthly sum. this wasy, we will get less spikes 

TODO Running average

In [ ]:
# group data into monthly sum
one_gauge_monthly_sums = one_gauge_joined.group_by_dynamic("DATETIME", every="1mo").agg(
    [pl.col("PRECIPITATION_GAUGE").sum(), pl.col("PRECIPITATION_GRID").sum()]
)

In [ ]:
one_gauge_monthly_sums = one_gauge_monthly_sums.with_columns(
    (pl.col("PRECIPITATION_GAUGE") - pl.col("PRECIPITATION_GRID")).alias(
        "rainfall_diff"
    )
)

In [ ]:
one_gauge_monthly_sums.head()

#### 🤨 Task: What is the mean monthly difference?
Replace the ??? below with your answer

In [ ]:
one_gauge_monthly_sums["rainfall_diff"].???

In [ ]:
one_gauge_monthly_sums.to_pandas().set_index("DATETIME")["rainfall_diff"].plot()
plt.axhline(color="k")

Hmm, is there any sort of trend in this data? Is it related to the number of gauges in the Upper Severn? That's a research question for another time...

##### ❓ Question for now: Is the nearest CEH-GEAR grid cell over or underestimating rainfall relative to the rain gauge? 

Answer: ???

#### 🤨 Task: Change the resolution of the time group_by below to:
1. 6 months
2. 1 year
3. Resolution of your chosing

Replace the ??? below with your answer

*Hint: find out online the correct syntax to group every 6 months and 1 year*
   

In [ ]:
one_gauge_joined.group_by_dynamic("DATETIME", every=???).agg(
    [
    pl.col("PRECIPITATION_GAUGE").sum(),
    pl.col("PRECIPITATION_GRID").sum()
    ]
)

??? = ???.with_columns(
    (pl.col("PRECIPITATION_GAUGE") - pl.col("PRECIPITATION_GRID")).alias("rainfall_diff")
)

???.to_pandas().set_index("DATETIME")['rainfall_diff'].plot()
plt.axhline(color='k')

#### 🤨 Task: Before moving on, go back to 3.1 and replace CHOSEN_GAUGE_ID with a new ID, then re-run all cells in 3.1

## 3.2 Repeating for multiple rain gauges
A lot of data research involves trial and error and repetition. It is always a good idea to get something working with a small subset before scaling up and applying your analysis to an entire dataset. In our case, we have code to compute a simple difference between a rain gauge and its nearest CEH-GEAR grid cell. Now let's scale that up to look at the entire catchment...

> In this section, we bring everything together, and scale up our research code   

Below are some functions with relatively descriptive names that will carry out all the analysis we saw in section 3.1.
In writing these, we have tried to stick to the Single Responsibility Principle as best we can (that is, one function does one thing)

In [ ]:
def calc_mean_diff_between_rain_gauge_and_nearest_grid_cell(gauge_id):
    """
    Calculate mean difference between a given rain gauge and the nearest CEH-GEAR grid cell
    """
    # 1. Load in gauge data and related metadata
    one_gauge = severn_rain_gauge_data_filtered.filter(pl.col("ID") == gauge_id)
    one_gauge_metadata = severn_rain_gauge_metadata.filter(pl.col("ID") == gauge_id)
    one_gauge_completeness = severn_rain_gauge_data_completeness.filter(
        pl.col("ID") == gauge_id
    )

    # 2. Get nearest CEH-GEAR grid cell
    one_gauge_grid_cell = get_nearest_gear_data_by_coords(
        easting=one_gauge_metadata["EASTING"], northing=one_gauge_metadata["NORTHING"]
    )

    # 3. Subset nearest grid cell data by start end date of gauge
    one_gauge_grid_cell = subset_gear_data_by_gauge_start_end_date(
        gear_data=one_gauge_grid_cell,
        start_date=one_gauge_completeness["start_date"].item(),
        end_date=one_gauge_completeness["end_date"].item(),
    )

    # 4. Convert data into polars format
    one_gauge_grid_cell_df = convert_gear_data_to_polars_df(one_gauge_grid_cell)

    # 5. Join to rain gauge data
    one_gauge_joined = one_gauge.join(one_gauge_grid_cell_df, on="DATETIME", how="left")
    one_gauge_joined = one_gauge_joined.rename({"PRECIPITATION": "PRECIPITATION_GAUGE"})

    # 6. Calculate daily rainfall differences
    one_gauge_joined = calculate_rainfall_difference(one_gauge_joined)
    return one_gauge_joined


def get_nearest_gear_data_by_coords(easting, northing):
    return severn_gear_daily.sel(x=easting, y=northing, method="nearest")


def subset_gear_data_by_gauge_start_end_date(gear_data, start_date, end_date):
    return gear_data.sel(time=slice(start_date, end_date))


def convert_gear_data_to_polars_df(gear_data):
    gear_data_rainfall_amount = gear_data["rainfall_amount"].data.flatten()
    gear_data_time = gear_data["time"].data.flatten()

    gear_data_df = pl.DataFrame(
        {"DATETIME": gear_data_time, "PRECIPITATION_GRID": gear_data_rainfall_amount}
    )
    gear_data_df = gear_data_df.with_columns(pl.col("DATETIME").cast(pl.Datetime("us")))
    return gear_data_df


def calculate_rainfall_difference(one_gauge_joined):
    one_gauge_joined = one_gauge_joined.with_columns(
        (pl.col("PRECIPITATION_GAUGE") - pl.col("PRECIPITATION_GRID")).alias(
            "rainfall_diff"
        )
    )
    one_gauge_joined = one_gauge_joined.drop_nulls()
    return one_gauge_joined

In [ ]:
CHOSEN_GAUGE_ID = 90358

In [ ]:
calc_mean_diff_between_rain_gauge_and_nearest_grid_cell(gauge_id=CHOSEN_GAUGE_ID)

Now, we have an easy one line to compute rainfall difference, we can loop through each gauge and compute some overall statistics

In [ ]:
# get all ids
all_unique_gauge_ids = severn_rain_gauge_data_filtered["ID"].unique().to_list()
all_unique_gauge_ids[:5] + ["..."]

In [ ]:
%%time
# Store all statistics in a dictionary
all_gauge_comparisons = []

for gauge_id in all_unique_gauge_ids:
    gauge_comparison = {}
    one_gauge_rainfall_diff = calc_mean_diff_between_rain_gauge_and_nearest_grid_cell(
        gauge_id=gauge_id
    )
    gauge_comparison["gauge_id"] = gauge_id
    gauge_comparison["stdev_diff"] = one_gauge_rainfall_diff["rainfall_diff"].mean()
    gauge_comparison["mean_diff"] = one_gauge_rainfall_diff["rainfall_diff"].std()
    gauge_comparison["max_diff"] = one_gauge_rainfall_diff["rainfall_diff"].max()
    gauge_comparison["min_diff"] = one_gauge_rainfall_diff["rainfall_diff"].min()
    all_gauge_comparisons.append(gauge_comparison)

In [ ]:
all_gauge_comparisons_df = pl.from_dicts(all_gauge_comparisons)
all_gauge_comparisons_df

There are lots of gauges, but a heatmap might be a quick visual way to check these values

In [ ]:
cols_to_plot = ["min_diff", "mean_diff", "stdev_diff", "max_diff"]
fig, ax = plt.subplots(1, figsize=(5, 9))
sns.heatmap(
    all_gauge_comparisons_df.select(cols_to_plot),
    cmap="RdBu_r",
    ax=ax,
    annot=True,
    fmt=".1f",
    cbar=False,
)
ax.set_xticklabels(cols_to_plot)
ax.set_ylabel("Gauge number")
ax.set_title("Summary of rain gauge difference from CEH-GEAR")

Of course, mean and standard deviation are relatively simple summary statistics, but you can imagine we can run more advanced statistics just as easily in this structure.

#### 🤨 Optional task: How correlated are each of the rain gauges to their nearest CEH-GEAR grid cell? 
If you cannot immediately think of how to do this, please move on


In [ ]:
## optional task

Moving on, Let's join this data back to the metadata so we can plot the gauges on a map

In [ ]:
severn_rain_gauge_metadata_w_comparison = severn_rain_gauge_metadata.join(
    all_gauge_comparisons_df, left_on="ID", right_on="gauge_id"
)

In [ ]:
severn_rain_gauge_metadata_w_comparison.head()

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 6), sharex=True, sharey=True)
ax.set_xlabel("Easting")
ax.set_ylabel("Northing")
sns.scatterplot(
    x="EASTING",
    y="NORTHING",
    hue="stdev_diff",
    palette='RdBu_r',
    data=severn_rain_gauge_metadata_w_comparison,
    ax=ax,
)
abermule_shp.plot(ax=ax, facecolor="none", alpha=0.8, linewidth=0.5)
bewdley_shp.plot(ax=ax, facecolor="none", alpha=0.8, linewidth=0.5)
dolwen_shp.plot(ax=ax, facecolor="none", alpha=0.8, linewidth=0.5)
plynlimon_shp.plot(ax=ax, facecolor="none", alpha=0.8, linewidth=0.5)

Hmm, there is not a clear spatial pattern in the difference between gauge and gridded data

## 3.3 Compute catchment averages and compare gridded rainfall to rain gauge observation
In this final example, we will compute catchment average rainfall for the smaller catchments in the Upper Severn (where the rain gauge network is more complete).

First, we need to make a mask of the smaller catchments. To do this, we have prepared a small function `make_region_hght_clip` that can clip the topography data, we can then use that to mask CEH-GEAR rainfall data. 

In [ ]:
# Make region clips using the Upper Severn catchment shapefiles
abermule_hght = data_utils.make_region_hght_clip(abermule_shp, hght_data=severn_hght)
dolwen_hght = data_utils.make_region_hght_clip(dolwen_shp, hght_data=severn_hght)
plynlimon_hght = data_utils.make_region_hght_clip(plynlimon_shp, hght_data=severn_hght)

Don't worry about understanding the code, but basically we can show what this is doing in the next plot...

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
severn_hght.plot(ax=axes[0], vmax=8.8, alpha=0.5, add_colorbar=False)
abermule_hght.plot(ax=axes[1], vmax=8.8, alpha=0.5, add_colorbar=False)

for ax in axes:
    abermule_shp.plot(ax=ax, facecolor="none", edgecolor="C2")
    ax.set_title("")
    ax.set_xlabel("")
    ax.set_ylabel("")

Next, we can mask the CEH-GEAR rainfall data to each region using another function `mask_regional_rainfall` which will create a subset based on the masks we just created. Again do not worry about the specifics, but if interested please look in the `data_utils.py` function. On a side note, it is always a good idea to move heavier functions outside of the notebook to keep things clean

In [ ]:
%%time
# Takes 60 seconds
abermule_mask_rainfall = data_utils.mask_region_rainfall(
    severn_gear_daily, abermule_hght
)
dolwen_mask_rainfall = data_utils.mask_region_rainfall(severn_gear_daily, dolwen_hght)
plynlimon_mask_rainfall = data_utils.mask_region_rainfall(
    severn_gear_daily, plynlimon_hght
)

Let's plot this, so we can see what it has done

In [ ]:
date_to_examine = "2018-01-01"  # feel free to look at other dates

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
severn_gear_daily.sel(time=date_to_examine)["rainfall_amount"].plot(
    ax=axes[0], vmax=12, cmap="Blues", cbar_kwargs={"label": "Daily rainfall (mm)"}
)

abermule_mask_rainfall.sel(time=date_to_examine)["rainfall_amount"].plot(
    ax=axes[1], vmax=12, cmap="Blues", cbar_kwargs={"label": "Daily rainfall (mm)"}
)

for ax in axes:
    abermule_shp.plot(ax=ax, facecolor="none", edgecolor="C2")
    ax.set_xlabel("")
    ax.set_ylabel("")

plt.suptitle(f"Date: {date_to_examine}", size=20)
axes[0].set_title("Rainfall across Severn region")
axes[1].set_title("Rainfall in Abermule catchment")

### 3.3.1 Calculate the 1990-2019 mean catchment and compare rainfall estimates across the 3 Upper Severn catchments 
Okay, we have the masks, now let's compute a 1990-2010 mean rainfall to see which parts of the catchment are wettest on average.

In [ ]:
catchments_shp_and_hght = {
    "Abermule": {"shp": abermule_shp, "hght": abermule_hght},
    "Dolwen": {"shp": dolwen_shp, "hght": dolwen_hght},
    "Plynlimon Flume": {"shp": plynlimon_shp, "hght": plynlimon_hght},
}

In [ ]:
%%time
abermule_rain = data_utils.mask_region_rainfall(
    severn_gear_daily, catchments_shp_and_hght["Abermule"]["hght"]
)



To compute the spatial mean, we need to run `.mean("time")`, so it will take a mean of the time coordinates (leaving only x, y) 

In [ ]:
%%time
abermule_mean_rain = abermule_rain.mean("time")["rainfall_amount"]

In [ ]:
fig, ax = plt.subplots(1)

abermule_mean_rain.plot(ax=ax)
catchments_shp_and_hght["Abermule"]["shp"].plot(
    ax=ax, facecolor="none", edgecolor=color_to_use, alpha=0.5
)
ax.set_xlim(280000, 330000)
ax.set_ylim(270000, 310000)

We loop through each of the catchments and plot them next

In [ ]:
%%time
fig, axes = plt.subplots(2, 2, figsize=(10, 7), sharex=True, sharey=True)

for ax, catchment, color_to_use in zip(
    axes.flatten(), catchments_shp_and_hght.keys(), ["C2", "C1", "C3"]
):
    region_mask_rain = data_utils.mask_region_rainfall(
        severn_gear_daily, catchments_shp_and_hght[catchment]["hght"]
    ).mean("time")["rainfall_amount"]
    region_mask_rain.plot(
        ax=ax,
        vmin=0,
        vmax=9,
        cmap="Blues",
        cbar_kwargs={"label": "Daily rainfall (mm)"},
    )
    catchments_shp_and_hght[catchment]["shp"].plot(
        ax=ax, facecolor="none", edgecolor=color_to_use, alpha=0.5
    )
    ax.set_title(f"{catchment}", size=16)
    ax.set_xlabel("Easting")
    ax.set_ylabel("Northing")
    ax.text(
        s=f"mean={region_mask_rain.mean():.2f}",
        x=306000,
        y=274000,
        size=12,
        color=color_to_use,
        alpha=0.9,
    )
axes[1][1].remove()
axes[1][0].text(
    s="CEH-GEAR gridded rainfall\n      (1990-2019 mean)",
    x=345000,
    y=290000,
    size=18,
    color="grey",
)
ax.set_xlim(280000, 330000)
ax.set_ylim(270000, 310000)

I wonder if the height profile has anything to do with spatial pattern of the mean rainfall?

### 3.3.2 Comparing catchment rainfall estimates through time
In 3.3.1, we look at the spatial catchment average, how about the time-average of the catchment rainfall

In [ ]:
# to keep RAM-use low, let's take a subset of data from 2010-2019
abermule_mask_rainfall_2010s = abermule_mask_rainfall.sel(time=slice("2010", "2019"))

To compute the time mean, we need to run `.mean(("x", "y"))`, so it will take a mean of the x, y coordinates (leaving only time) 

In [ ]:
%%time
abermule_catchment_mean_rainfall_2010s = abermule_mask_rainfall_2010s[
    "rainfall_amount"
].mean(("x", "y"))

In [ ]:
abermule_catchment_mean_rainfall_2010s.plot()

Okay, what about the rain gauges within those catchments, what do they estimate?  
Next, we will compare the catchment average of the CEH-GEAR to the average of the rain gauges in the catchment. For this purpose, we will convert the metadata into a geodataframe. This will allow us to compute a [Point in Polygon](https://en.wikipedia.org/wiki/Point_in_polygon) operation, where the 'points' are the rain gauge location, and the 'polygons' are the catchment boundaries i.e. `abermule_shp` 

In [ ]:
# Create a geometry column for the metadata gdf based on the X,Y coordiantes of rain gauges
geometry = [
    shapely.geometry.Point(xy)
    for xy in zip(
        severn_rain_gauge_metadata_w_comparison["EASTING"],
        severn_rain_gauge_metadata_w_comparison["NORTHING"],
    )
]

In [ ]:
# Create a GeoDataFrame using that geometry
severn_rain_gauge_metadata_gdf = gpd.GeoDataFrame(
    severn_rain_gauge_metadata_w_comparison.to_pandas(),
    geometry=geometry,
    crs="EPSG:27700",
)

In [ ]:
severn_rain_gauge_metadata_gdf.head()

##### ❓ Question: What data type is `severn_rain_gauge_metadata_gdf`?
**Answer:** ???

Below we use a `.contains()` method to compute Point in Polygon operation i.e. is a given rain gauge within the abermule shapefile

In [ ]:
abermule_metadata = severn_rain_gauge_metadata_gdf.loc[
    severn_rain_gauge_metadata_gdf.apply(
        lambda row: abermule_shp.contains(row["geometry"]), axis=1
    ).values
]

Let's view the gauges within one of the Upper Severn catchments: Abermule

In [ ]:
fig, ax = plt.subplots(1)
abermule_metadata.plot(ax=ax)
abermule_shp.plot(ax=ax, facecolor="none")

Let's get the rain gauge data for those Abermule gauge IDs

In [ ]:
abermule_rain_gauge_data = severn_rain_gauge_data_filtered.filter(
    pl.col("ID").is_in(abermule_metadata["ID"].to_list())
)

In the next step, we pivot the Abermule rain gauge data so that each column refers to a different gauge. Rather than describing how this works, have a look at the output data

In [ ]:
abermule_rain_gauge_data_pivot = abermule_rain_gauge_data.pivot(
    on="ID", index="DATETIME", values="PRECIPITATION"
)

# Sort the pivot table by time (for plotting purporses)
abermule_rain_gauge_data_pivot = abermule_rain_gauge_data_pivot.sort(by="DATETIME")

In [ ]:
abermule_rain_gauge_data_pivot.head()

We will quickly visualise the 6 month rolling mean for these Abermule rain gauges. Don't worry about the code below

In [ ]:
abermule_rain_gauge_data_6month = abermule_rain_gauge_data_pivot.group_by_dynamic(
    "DATETIME", every="6mo"
).agg([pl.col(col).mean() for col in abermule_rain_gauge_data_pivot.columns[1:]])

In [ ]:
abermule_rain_gauge_data_6month.to_pandas().set_index("DATETIME").plot()

Finally, we will get data from 2010-2019

In [ ]:
abermule_rain_gauge_data_2010s = abermule_rain_gauge_data_pivot.filter(
    (pl.col("DATETIME") >= pl.datetime(year=2010, month=1, day=1))
    & (pl.col("DATETIME") < pl.datetime(year=2020, month=1, day=1))
)

And compute a mean of all the gauges in the Abermule catchment

In [ ]:
abermule_rain_gauge_data_2010s = abermule_rain_gauge_data_2010s.with_columns(
    pl.mean_horizontal(pl.all().exclude("DATETIME")).alias("gauge mean")
)

In [ ]:
abermule_rain_gauge_data_2010s["gauge mean"].plot.line()

Here's the catchment-wide mean from CEH-GEAR again:

In [ ]:
abermule_catchment_mean_rainfall_2010s.plot()

hmm, they look similar, but are they? See additional task 1 if you would like to continue

## ❗❗ Additional tasks ❗❗  
Feel free to stop at this point, but below are some additional and more advanced topics and tasks requiring more of your own input. We will provide help.  
Select any of the following:

*Task 1. Compare catchment-wide rainfall estimates (grid vs gauge)* 🟢  
*Task 2. Does rainfall disproportionately fall at higher altitudes before floods?* 🔴  
*Task 3. A case study of an unseen rain gauge in the Upper Severn* 🟡   
*Task 4. Looking at future projections of precipitation in the Severn using CHESS-SCAPE* 🔴

#### Task Difficulty:
- 🟢 -> easy, you should have all of the code to do this
- 🟡 -> moderate, will require a little more work and a better understanding of the data
- 🔴 -> hard, requires lots of independent thought



## 🟢 Additonal Task 1: Compare catchment-mean rainfall estimates CEH-GEAR vs Rain Gauge Observation
We already should have catchment-mean rainfall for the Abermule catchment loaded in (in section 3.3.2), so you choose to do this task just below. It may be a good learning opportunity to develop your own statistical analysis. Perhaps by running a linear regression?

*Hint: you could import the `scipy.stats` library to run statistics and `numpy` gives you access to regression*  
*Hint: Consider using (monthly) running means to plot data*  

In [ ]:
abermule_catchment_mean_rainfall_2010s

In [ ]:
abermule_rain_gauge_data_2010s

## 🔴 Additional Task 2. Does rainfall disproportionately fall at higher altitudes during floods?
> Note: please do this in a new notebook file

If you need a hint, check out some of the plots and notebooks in: https://github.com/NERC-CEH/FDRI-high-altitude-rainfall-and-floods

In [ ]:
# Remember that the heaviest rainfall is often a day or two before the flood event
EXAMPLE_FLOOD_DAYS_IN_SEVERN = [
    "1984-11-13",
    "1986-12-16",
    "1986-12-19",
    "1988-03-16",
    "1988-03-20",
    "1990-03-01",
    "1990-12-27",
    "1994-01-04",
    "1994-01-14",
    "1994-02-28",
    "1994-04-05",
    "1994-04-10",
    "1995-01-22",
    "1995-12-24",
    "1996-02-13",
    "2000-09-28",
    "2001-02-13",
    "2002-01-28",
    "2002-11-15",
    "2007-01-01",
    "2008-03-17",
    "2008-11-11",
    "2012-04-30",
    "2013-02-15",
    "2020-01-15",
]

In [ ]:
# A function for masking rainfall data by height
def mask_region_rainfall_by_hght(rainfall_data, region_hght, threshold):
    region_hght_mask = region_hght / region_hght.where(region_hght > threshold)
    return rainfall_data * region_hght_mask.data

## 🟡 Additional Task 3. Case Study: Examining unseen rain gauge data in the Upper Severn
> Note: please do this in a new notebook file

The CEH-GEAR gridded product was produced on the rain gauge data we have used in this practical. As such, there is a direct spill of information between them. On the JASMIN object store there is an additional "unseen" rain gauge near Carreg Wen within the Plynlimon catchment. 

#### Some research questions:
- How closely do the CEH grid cells around Carreg Wen estimate rainfall?
- Has the gridded product under or over-estimated rainfall in the days before a flood?

*Hint: If you really need a hint, you can check this [notebook](https://github.com/NERC-CEH/FDRI-comparing-rainfall-data-in-upper-severn/blob/main/notebooks/Carreg_wen_case_study.ipynb)*

In [ ]:
carreg_wen_daily = pl.read_csv(
    "s3://rain-gauge/carreg_wen_daily_rainfall.csv",
    storage_options={"endpoint_url": "https://fdri-o.s3-ext.jc.rl.ac.uk", "anon": True},
    try_parse_dates=True,
)
carreg_wen_daily.head()

In [ ]:
severn_rain_gauge_metadata.filter(pl.col("NAME") == "CARREG WEN")

In [ ]:
fig, ax = plt.subplots(1)
abermule_metadata.plot(ax=ax)
abermule_shp.plot(ax=ax, facecolor="none")
plynlimon_shp.plot(ax=ax, facecolor="none")
ax.scatter(x=282900, y=288500)
ax.text(s="Carreg Wen", x=284700, y=288200, c="C1")
ax.legend(["Gauges used in CEH-GEAR"])

In [ ]:
# Remember that the heaviest rainfall is often a day or two before the flood event
EXAMPLE_FLOOD_DAYS_IN_SEVERN = [
    "1984-11-13",
    "1986-12-16",
    "1986-12-19",
    "1988-03-16",
    "1988-03-20",
    "1990-03-01",
    "1990-12-27",
    "1994-01-04",
    "1994-01-14",
    "1994-02-28",
    "1994-04-05",
    "1994-04-10",
    "1995-01-22",
    "1995-12-24",
    "1996-02-13",
    "2000-09-28",
    "2001-02-13",
    "2002-01-28",
    "2002-11-15",
    "2007-01-01",
    "2008-03-17",
    "2008-11-11",
    "2012-04-30",
    "2013-02-15",
    "2020-01-15",
]

## 🔴 Additional Task 4. Looking at future projections of precipitation using CHESS-SCAPE
> Note: please do this in a new notebook file

CHESS-SCAPE provides future projections of precipitation across the UK. This rich dataset enables a wide range of climate-related research questions. You can explore and analyze these data using code similar to what's shown in this notebook. 

#### Example research questions
- How is precipitation projected to change across the Upper Severn over time?
- Which catchments are expected to experience the greatest increase in extreme rainfall events?

> If you have a problem loading in the zarr file, please check you are using zarr>=3.0.8 and you have cftime installed

In [ ]:
# We are accessing TASMAX & PRCPT for the Ensemble member #01 from the catalogue
fs = fsspec.filesystem(
    "s3",
    asynchronous=True,
    anon=True,
    endpoint_url="https://chess-scape-o.s3-ext.jc.rl.ac.uk",
)
pr_zstore = zarr.storage.FsspecStore(
    fs, path="ens01-year100kmchunk/pr_01_year100km.zarr"
)

chess_pr = xr.open_zarr(
    pr_zstore, decode_times=True, decode_cf=True, consolidated=False
)

In [ ]:
chess_pr

# Additional Reading

- <https://github.com/NERC-CEH/FDRI-comparing-rainfall-data-in-upper-severn/tree/main>  

- <https://github.com/NERC-CEH/FDRI-high-altitude-rainfall-and-floods>